In [ ]:
import sounddevice as sd
import soundfile as sf
import speech_recognition as sr
import pyaudio
import wave
import time
import whisper
from email.message import EmailMessage
import ssl
import smtplib
import pyautogui
import webbrowser
import cv2
import pymongo 

print('Datos personales\n')
nombre_usuario = input(str('Nombre: '))
apellidos_usuario = input(str('Apellidos: '))
email = input('Correo electronico: ')
phone_number = input("Numero de telefono: ")
print(' ')
print('Datos de emergencia\n')
phone_helpers = input("Telefono de emergencia: ")
email_ayuda = input(str('Correo electronico de emergencia: '))
keyword = input(str('Palabra clave: '))
print('\n\n')

# Crear el diccionario con la palabra y el texto para email y whatsapp (info reducida)
diccionario = {'Nombre:      ': nombre_usuario,
               'Apellidos    ': apellidos_usuario,
               'Telf:        ': phone_number,
               'Latitud:     ': 40.3923074,
               'Longitud:    ': -3.7000049,
               'Ubicación:   ': 'PASEO CHOPERA 14, 28045, Madrid ',
              }


# Configuración de parámetros
duration_video = 15
duration = 30  # Duración de la grabación en segundos
sample_rate = 44100  # Tasa de muestreo en Hz
channels = 2  # Número de canales de audio (estéreo)
output_path = "/Users/JGR/Desktop/SYL/audio/"
input_path = "/Users/JGR/Desktop/SYL/texto/"



# Función para manejar la activación de la palabra clave
def syl_nucleo():
    # Inicializar el dispositivo de grabación de audio
    audio = pyaudio.PyAudio()

    # Configurar los parámetros del audio
    formato = pyaudio.paInt16
    canales = channels
    tasa_muestreo = sample_rate
    frames_por_buffer = 1024

    # Crear una lista para almacenar los frames de audio
    frames = []

    # Función para capturar los frames de audio
    def capturar_frames(in_data, frame_count, time_info, status):
        frames.append(in_data)
        return (in_data, pyaudio.paContinue)

    # Crear el stream de audio para la grabación
    stream = audio.open(format=formato,
                        channels=canales,
                        rate=tasa_muestreo,
                        input=True,
                        frames_per_buffer=frames_por_buffer,
                        stream_callback=capturar_frames)


    # Iniciar la grabación de audio
    stream.start_stream()

    # Esperar durante la duración especificada
    time.sleep(duration)

    # Detener la grabación de audio
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Guardar el archivo de audio en formato WAV
    filename = output_path + time.strftime("%Y%m%d-%H%M%S") + ".wav"
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(audio.get_sample_size(formato))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()
    video_capture = cv2.VideoCapture(0)  # 0 represents the default camera
    # Define the output file path with a unique filename
    fourcc = cv2.VideoWriter_fourcc(*"mp4v") 
    output_directory = '/Users/JGR/Desktop/SYL/camara/'
    current_time = time.strftime("%Y%m%d-%H%M%S")
    name_video = (current_time + 'mp4')
    output_file_path = output_directory + current_time + '.mp4'
    output_file = cv2.VideoWriter(output_file_path, fourcc, 20.0, (640, 480))
    # Define the codec and create a VideoWriter object
     # Use appropriate codec based on your system
    fps = 15.5  # Frames per second
    frame_width = int(video_capture.get(3))  # Width of the captured frames
    frame_height = int(video_capture.get(4))  # Height of the captured frames
    output_file = cv2.VideoWriter(output_file_path, fourcc, fps, (frame_width, frame_height))

    # Start recording
    start_time = time.time()
    while (time.time() - start_time) < duration_video:
        # Read each frame from the camera
        ret, frame = video_capture.read()

        # Display the resulting frame
        cv2.imshow('Video', frame)

        # Write the frame to the output file
        output_file.write(frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the resources
    video_capture.release()
    output_file.release()
    cv2.destroyAllWindows()
    # Forcefully close the OpenCV window
    cv2.waitKey(1)

    
 # WHISPER


    model = whisper.load_model("base")
    result = model.transcribe(filename, fp16=False)
    

    print("\n")
    text = result["text"]
    ref_number = (filename[-19:-4])
    file_name = ref_number + '.txt'
    file_path = input_path + file_name
    
    
   
    
    
   # condicionales 
    palabras = ['maltrates', 'no me pegues', 'matar', 'te voy a matar', 'me estan pegando', 'dejame en paz']
    palabras1 = ['gilipollas', 'maricón', 'inútil', 'lesbiana']
    palabras2 = ['me estan robando', 'me han robado', 'me quieren robar']
    palabras3 = ['residencia', 'ayuda', 'no me insultes', 'dejame tranquilo', 'dejame tranquila']
    
    for clave, valor in diccionario.items():
        print(clave, valor)
        dicc = diccionario.items()    
        

    if any(palabra in text for palabra in palabras):
        print('''

    Parece ser que está sufrindo daños físicos, pongase en contacto lo antes posible.
    
    Relato:
        ''')
        resultado = "Parece ser que está sufrindo daños físicos, pongase en contacto lo antes posible. Relato:"

    elif any(palabra in text for palabra in palabras1):
        print('''

    Parece ser que está sufrindo abuso, pongase en contacto lo antes posible.
    
    Relato:
        ''')
        resultado = "Parece ser que está sufrindo abuso, pongase en contacto lo antes posible. Relato:"

    elif any(palabra in text for palabra in palabras2):
        print('''

    Parece ser que está sufriendo un robo, por favor pongase en contacto lo antes posible
    
    Relato:
        ''')
        resultado = "Parece ser que está sufriendo un robo, por favor pongase en contacto lo antes posible. Relato:"

    elif any(palabra in text for palabra in palabras3):
        print('''

    Parece ser que necesita ayuda, por favor! pongase en contacto lo antes posible.
    
    Relato:
        ''')
        resultado = "Parece ser que necesita ayuda, por favor! pongase en contacto lo antes posible. Relato:"

    else:
        print('''

    Pongase en contacto por favor lo antes posible
    
    Relato:
        ''')
        resultado = "Pongase en contacto por favor lo antes posible. Relato:"
        

    with open(file_path, 'w') as file:
        file.write(text)
    print('')
    print(text)
    print('')
    print("Audio guardado, ref: ", ref_number)
    print("Video guardado, ref: ", current_time)

    
     # EMAIL 
    sender = '######################' # sustituir por tu correo electronico
    email_password = '########################' # sustituir por la contraseña de tu correo electronico
    rec_email = email_ayuda
    subject = 'ALERTA'

    body = text
    em = EmailMessage()
    em['From'] = sender
    em['To'] = rec_email
    em['Subject'] = subject
    em['Attachment'] = name_video
    contenido = f'{dicc}\n{resultado}\n{body}' 
    em.set_content(contenido)  
    with open(output_file_path, 'rb') as file:
        em.add_attachment(file.read(), maintype='video', subtype='mp4', filename= name_video) 
        context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(sender, email_password)
        smtp.sendmail(sender, rec_email, em.as_string())
        print('''
Email enviado correctamente.

Whatsapp enviado correctamente.

''')        

        
    # WHATSAPP
    
    webbrowser.open('https://web.whatsapp.com/send?phone=' + phone_number)

    # Esperar unos segundos para que se cargue WhatsApp Web y puedas escanear el código QR
    time.sleep(10)

    # Esperar unos segundos adicionales para asegurarse de que se haya cargado completamente la página
    time.sleep(10)

    # Simular el envío del mensaje utilizando pyautogui
    pyautogui.typewrite(contenido)
    pyautogui.press('enter')
        
    #MongoDB - DATABASE

    client = pymongo.MongoClient("mongodb://localhost:27017")

    # Seleccion de Base De Datos
    db = client["SYL"]
    collection = db["SYL.USERS"]

    diccionario_mongo = {

        'User_name': nombre_usuario,
        'User_last_name': apellidos_usuario,
        'Telefono': phone_number,
        'Email' : email,
        'Telefono_persona_urgencia': phone_helpers,
        'Email_persona_urgencia': email_ayuda,
        'Latitud': 40.3923074,
        'Longitud': -3.7000049,
        'Ubicacion': 'PASEO CHOPERA 14, 28045, Madrid ',
        'Grabedad': resultado,
        'Relato': body,   
    }
    
    collection.insert_one(diccionario_mongo)
    
    
    
# Crear un reconocedor de voz
recognizer = sr.Recognizer()

# Crear una instancia de micrófono
microphone = sr.Microphone()

print("Escuchando...\n")

# Continuamente escuchar hasta que se detecte la palabra clave
while True:
    # Escuchar el audio con el micrófono
    with microphone as source:
        audio = recognizer.listen(source)

    try:
        # Convertir el audio a texto
        reconocer_palabra_clave = recognizer.recognize_google(audio, language="es-ES")

        # Verificar si se detectó la palabra clave
        if keyword.lower() in reconocer_palabra_clave.lower():
            print("Grabando...\n")
            syl_nucleo()
            break

    except sr.UnknownValueError:
        pass
    


Datos personales

